In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import random


In [ ]:
model = tf.keras.models.load_model("floodmap_generator.keras")

In [ ]:
def heightmap(height):

  if not os.path.exists(height):
    print(f"Warning: Segmentation map file not found: {height}")

  else:
    h_map = cv2.imread(height)

  # Extracting height and width to resize the image and maintain aspect ratio

    h, w = h_map.shape[:2]
    ratio = 256/w
    resized_h = cv2.resize(h_map,(256,int(h*ratio)))

    height_map = cv2.cvtColor(resized_h, cv2.COLOR_BGR2GRAY)

    max_h = np.unravel_index(np.argmax(height_map), height_map.shape)

  # Setting a height rating value

    height_rating = np.zeros(height_map.shape,dtype = 'float32')
    for i in range(0,height_map.shape[0]):
      for j in range(0,height_map.shape[1]):
        height_rating[i][j] = height_map[i][j]/height_map[max_h[0],max_h[1]]

    return height_rating

In [ ]:
def segmentationmap(segmentation):

  if not os.path.exists(segmentation):
    print(f"Warning: Segmentation map file not found: {segmentation}")


  else:
    s_map = cv2.imread(segmentation,cv2.IMREAD_COLOR)

  # Extracting height and width to resize the image and maintain aspect ratio

    h, w = s_map.shape[:2]
    ratio = 256/w
    resized_s = cv2.resize(s_map,(256,int(h*ratio)))

  # Creating an array for the segmentation map (shape = 256x256x3)

    seg_map = cv2.cvtColor(resized_s, cv2.COLOR_BGR2RGB)

  # Creating an array to store the terrain data of each pixel

    terrain = np.zeros((seg_map.shape[0], seg_map.shape[1]), dtype='float32')
    for i in range (0,seg_map.shape[0]):
      for j in range (0,seg_map.shape[1]):
        if np.array_equal(seg_map[i,j], [128,0,128]):# Village Houses
          terrain[i][j] = 8
        elif np.array_equal(seg_map[i,j], [17,141,215]):# Water
          terrain[i][j] = 1
        elif np.array_equal(seg_map[i,j], [225,227,155]):# Grasslands
          terrain[i][j] = 2
        elif np.array_equal(seg_map[i,j], [127,173,123]):# Forest
          terrain[i][j] = 3
        elif np.array_equal(seg_map[i,j], [185,122,87]):# Hills
          terrain[i][j] = 4
        elif np.array_equal(seg_map[i,j], [230,200,181]):# Desert
          terrain[i][j] = 5
        elif np.array_equal(seg_map[i,j], [150,150,150]):# Mountains
          terrain[i][j] = 6
        elif np.array_equal(seg_map[i,j], [193,190,175]):# Tundra
          terrain[i][j] = 7

    return seg_map,terrain

In [ ]:
# Finding the approximate area the village occupies and extracting the corresponding regions from all the maps
def find_village(data):
  village = 0
  min_x,min_y = 256,256
  max_x,max_y = 0,0
  for i in range(data.shape[0]):
    for j in range(data.shape[1]):
      if data[i][j][1] == 8:
        village = 1
        min_x = min(min_x, j)
        min_y = min(min_y, i)
        max_x = max(max_x, j)
        max_y = max(max_y, i)

  top_left = (min_x,min_y)
  bottom_right = (max_x,max_y)

  if village == 1:
    for k in range(0,7):
      if max_x<data.shape[0]:
        max_x += 1
      if max_y<data.shape[1]:
        max_y += 1
      if min_x>0:
        min_x -= 1
      if min_y>0:
        min_y -= 1
    top_left_l = (min_x,min_y)
    bottom_right_l = (max_x,max_y)

    rectangle = cv2.rectangle(seg_map, top_left,bottom_right,(0, 255, 0), 1)
    rectangle_2 = cv2.rectangle(seg_map, top_left_l,bottom_right_l,(0, 255, 0), 1)
    rectangle_3 = cv2.rectangle(seg_map,(min_x - 20,min_y - 20),(max_x + 20,max_y + 20),(0, 255, 0), 1)

    rectangle_4 = cv2.rectangle(flood_map, top_left,bottom_right,(0, 255, 0), 1)
    rectangle_5 = cv2.rectangle(flood_map, top_left_l,bottom_right_l,(0, 255, 0), 1)
    rectangle_6 = cv2.rectangle(flood_map,(min_x - 20,min_y - 20),(max_x + 20,max_y + 20),(0, 255, 0), 1)

  seg_map_copy = seg_map
  flood_map_copy = flood_map
  height_rating_copy = height_rating
  data_copy = data

  return min_x,min_y,max_x,max_y,seg_map_copy,flood_map_copy,height_rating_copy,data_copy,village

In [ ]:
def flood_reducer(image,jitter=0.4):
    height, width = image.shape[0], image.shape[1]

    center = (width // 2, height // 2)

    radius = min(width, height) // 4

    circle_points = []

    for y in range(height):
        for x in range(width):

            dx = x - center[0]
            dy = y - center[1]
            distance = np.sqrt(dx**2 + dy**2)

            angle = np.arctan2(dy, dx)
            random.seed(int(angle * 1000))
            varied_radius = radius * (1 + (random.random() * 2 - 1) * jitter)

            if distance <= varied_radius:
                circle_points.append((x, y))

    return circle_points

In [ ]:
height_input = input("Enter the height map path: ")
seg_input = input("Enter the segmentation map path:")

height_rating = heightmap(height_input)
seg_map,terrain = segmentationmap(seg_input)

data = np.zeros((terrain.shape[0],terrain.shape[1],2), dtype = 'float32')
for i in range (0,terrain.shape[0]):
  for j in range (0,terrain.shape[1]):
    data[i][j][0] = height_rating[i][j]
    data[i][j][1] = terrain[i][j]

reshaped_data = data.reshape(1, data.shape[0], data.shape[1], data.shape[2])

# Getting the predicted flood map

flood_map = model.predict(reshaped_data)
flood_map = flood_map.reshape(256,256,3)

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.imshow(flood_map)
plt.title('Flood Map')
plt.colorbar()
plt.show()

In [ ]:
min_x,min_y,max_x,max_y,seg_map_copy,flood_map_copy,height_rating_copy,data_copy,village = find_village(data)
if village == 1:
  x1,x2,y1,y2 = min_x,max_x,min_y,max_y
  for a in range(0,20):
    if x1>0:
      x1 -= 1
    if y1>0:
      y1 -= 1
    if x2<data.shape[0]:
      x2 += 1
    if y2<data.shape[1]:
      y2 += 1

  if min_y < max_y and min_x < max_x:
    village_region = seg_map_copy[min_y:max_y, min_x:max_x]
    village_region2 = seg_map_copy[y1:y2, x1:x2]

    flood_region = flood_map_copy[min_y:max_y, min_x:max_x]
    flood_region2 = flood_map_copy[y1:y2, x1:x2]
    height_region = height_rating_copy[min_y:max_y, min_x:max_x]
    height_region2 = height_rating_copy[y1:y2, x1:x2]
    data_region = data_copy[min_y:max_y, min_x:max_x]
    data_region2 = data_copy[y1:y2, x1:x2]

    plt.figure(figsize=(10, 5))
    plt.subplot(121)
    plt.imshow(village_region2)
    plt.title('Village Region')
    plt.colorbar()
    plt.show()

    avg_f = np.mean(flood_region,axis=(0, 1))
    avg_f *= 1.003
  village_top = village_region2[0:27,0:village_region2.shape[1]]
  flood_top = flood_region2[0:27,0:village_region2.shape[1]]
  height_top = height_region2[0:27,0:village_region2.shape[1]]
  avg_top_left = np.mean(flood_top[0:27,0:int(village_region2.shape[1]/2)],axis=(0, 1))
  avg_top_right = np.mean(flood_top[0:27,int(village_region2.shape[1]/2):village_region2.shape[1]],axis=(0, 1))

  village_left = village_region2[0:village_region2.shape[0],0:27]
  flood_left = flood_region2[0:village_region2.shape[0],0:27]
  height_left = height_region2[0:village_region2.shape[0],0:27]
  avg_left_top = np.mean(flood_left[0:int(village_region2.shape[0]/2),0:27],axis=(0, 1))
  avg_left_bottom = np.mean(flood_left[int(village_region2.shape[0]/2):village_region2.shape[0],0:27],axis=(0, 1))

  village_right = village_region2[0:village_region2.shape[0],village_region2.shape[1]-26:village_region2.shape[1]]
  flood_right = flood_region2[0:village_region2.shape[0],village_region2.shape[1]-26:village_region2.shape[1]]
  height_right = height_region2[0:village_region2.shape[0],village_region2.shape[1]-26:village_region2.shape[1]]
  avg_right_top = np.mean(flood_right[:int(flood_right.shape[0]/2),0:flood_right.shape[1]],axis=(0, 1))
  avg_right_bottom = np.mean(flood_right[int(flood_right.shape[0]/2):,0:flood_right.shape[1]],axis=(0, 1))

  village_bottom = village_region2[village_region2.shape[0]-27:village_region2.shape[0],0:village_region2.shape[1]]
  flood_bottom = flood_region2[village_region2.shape[0]-27:village_region2.shape[0],0:village_region2.shape[1]]
  height_bottom = height_region2[village_region2.shape[0]-27:village_region2.shape[0],0:village_region2.shape[1]]
  avg_bottom_left = np.mean(flood_bottom[0:flood_bottom.shape[0],0:int(village_region2.shape[1]/2)],axis=(0, 1))
  avg_bottom_right = np.mean(flood_bottom[0:flood_bottom.shape[0],int(village_region2.shape[1]/2):village_region2.shape[1]],axis = (0, 1))

else:
  print("Village region not found or is empty.")

count = 0
if village == 1:
  if avg_top_left[2] > avg_f[2]:
    j = 6
    while j < int(village_region.shape[1]/2):
      group = random.randint(2,5)

      if j>=village_region.shape[1]*2/5:
        i = 2
      elif j>village_region.shape[1]/5:
        i = 4
      elif j>0:
        i = 6

      for k in range(group):
        if j + k < int(village_region.shape[1]/2):
            if data_region[i][j+k][1]!=1:
              height_region[i][j+k] = 0.20
              village_region[i][j+k] = [255, 255, 255]
              count += 1

      gap = random.randint(2,4)
      j += group + gap

  if avg_top_right[2] > avg_f[2]:
    j = int(village_region.shape[1]/2)
    while j < village_region.shape[1] - 6:
      group = random.randint(2,5)

      if j<village_region.shape[1]*3/5:
        i = 2
      elif j<village_region.shape[1]*4/5:
        i = 4
      elif j<village_region.shape[1]:
        i = 6

      for k in range(group):
        if j + k < village_region.shape[1]-6:
          if data_region[i][j+k][1]!=1:
            height_region[i][j+k] = 0.20
            village_region[i][j+k] = [255, 255, 255]
            count += 1

      gap = random.randint(2,4)
      j += group + gap

  if avg_bottom_left[2] > avg_f[2]:
    j2 = 6
    while j2 < int(village_region.shape[1]/2):
      group = random.randint(2,5)

      if j2>=village_region.shape[1]*2/5:
        i2 = village_region.shape[0] - 2
      elif j2>village_region.shape[1]/5:
        i2 = village_region.shape[0] - 4
      elif j2>0:
        i2 = village_region.shape[0] - 6

      for k in range(group):
        if j2 + k < int(village_region.shape[1]/2):
          if data_region[i2][j2+k][1]!=1:
            height_region[i2][j2+k] = 0.20
            village_region[i2][j2+k] = [255, 255, 255]
            count += 1

      gap = random.randint(2,4)
      j2 += group + gap

  if avg_bottom_right[2] > avg_f[2]:
    j2 = int(village_region.shape[1]/2)
    while j2 < village_region.shape[1] - 6:
      group = random.randint(2,5)

      if j2<village_region.shape[1]*3/5:
        i2 = village_region.shape[0] - 2
      elif j2<j<village_region.shape[1]*4/5:
        i2 = village_region.shape[0] - 4
      elif j2<village_region.shape[1]:
        i2 = village_region.shape[0] - 6

      for k in range(group):
        if j2 + k < int(village_region.shape[1]):
          if data_region[i2][j2+k][1]!=1:
            height_region[i2][j2+k] = 0.20
            village_region[i2][j2+k] = [255, 255, 255]
            count += 1

      gap = random.randint(2,4)
      j2 += group + gap

  if avg_left_top[2] > avg_f[2]:
    i = 6
    while i < int(village_region.shape[0]/2):
      group = random.randint(2,5)

      if i>=village_region.shape[0]*2/5:
        j = 2
      elif i>village_region.shape[0]/5:
        j = 4
      elif i>0:
        j = 6

      for k in range(group):
        if i + k < int(village_region.shape[0]/2):
          if data_region[i+k][j][1]!=1:
            height_region[i+k][j] = 0.20
            village_region[i+k][j] = [255, 255, 255]
            count+=1

      gap = random.randint(2,4)
      i += group + gap

  if avg_left_bottom[2] > avg_f[2]:
    i = int(village_region.shape[0]/2)
    while i < village_region.shape[0]-6:
      group = random.randint(2,5)

      if i<village_region.shape[0]*3/5:
        j = 2
      elif i<village_region.shape[0]*4/5:
        j = 4
      elif i<village_region.shape[0]:
        j = 6

      for k in range(group):
        if i + k < village_region.shape[0]-6:
          if data_region[i+k][j][1]!=1:
            height_region[i+k][j] = 0.20
            village_region[i+k][j] = [255, 255, 255]
            count+=1

      gap = random.randint(2,4)
      i += group + gap

  if avg_right_top[2] > avg_f[2]:
    i2 = 6
    while i2 < int(village_region.shape[0]/2):
      group = random.randint(2,5)

      if i2>=village_region.shape[0]*2/5:
        j2 = village_region.shape[1] - 2
      elif i2>village_region.shape[0]/5:
        j2 = village_region.shape[1] - 4
      elif i2>0:
        j2 = village_region.shape[1] - 6

      for k in range(group):
        if i2 + k < int(village_region.shape[0]/2):
          if data_region[i2+k][j2][1]!=1:
            height_region[i2+k][j2] = 0.20
            village_region[i2+k][j2] = [255, 255, 255]
            count+=1


      gap = random.randint(2,4)
      i2 += group + gap

  if avg_right_bottom[2] > avg_f[2]:
    i2 = int(village_region.shape[0]/2)
    while i2 < village_region.shape[0]-6:
      group = random.randint(2,5)

      if i2<village_region.shape[0]*3/5:
        j2 = village_region.shape[1] - 2
      elif i2<village_region.shape[0]*4/5:
        j2 = village_region.shape[1] - 4
      elif i2<village_region.shape[0]:
        j2 = village_region.shape[1] - 6

      for k in range(group):
        if i2 + k < village_region.shape[0]-6:
          if data_region[i2+k][j2][1]!=1:
            height_region[i2+k][j2] = 0.20
            village_region[i2+k][j2] = [255, 255, 255]
            count+=1


      gap = random.randint(2,4)
      i2 += group + gap

points = flood_reducer(village_region)
for g,h in points:
  if g < flood_region.shape[0] and h < flood_region.shape[1]:
    flood_region[g][h][0] -= count/2000
    flood_region[g][h][1] -= count/1500
    flood_region[g][h][2] += count/1000


In [ ]:
if min_y < max_y and min_x < max_x:
  plt.figure(figsize=(10, 5))
  plt.subplot(121)
  plt.imshow(village_region)
  plt.title('Architecture Placement')
  plt.colorbar()
  plt.show()

plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.imshow(flood_map_copy,cmap = 'Blues')
plt.title('New Flood Map')
plt.colorbar()
plt.show()

plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.imshow(seg_map_copy)
plt.title('New Segmentation Map')
plt.colorbar()
plt.show()